In [12]:
#Importing useful packages
import sqlite3
import csv
import pandas as pd
from datetime import datetime

#Returns a Dataframe containing a sample of size "number" from the specified month and year
def get_sample(year,month,number):
    #Error Checking
    if number < 0:
        print("Bad Number Value\n")
        return
    if year < 2012 | year > 2014:
        print("Bad year value\n")
        return
    
    #Change to directory of your database!
    conn = sqlite3.connect(r'E:\Summer Project\yellow_taxi_'+str(year)+'.sqlite')
    c = conn.cursor()
    #For small samples (less than 10000) adding them this way is fastest
    if number<10000:
        sample = pd.read_sql_query('select * from '+str(month)+
                                   ' where rowid = (abs(random()) % (select (select max(rowid) from january)+1))',conn)
        for i in range(number-1):
            sample = sample.append(pd.read_sql_query('select * from '+str(month)+' where rowid = (abs(random()) % (select (select max(rowid) from january)+1))',conn))#row,ignore_index=True)
    else:
        #This is the fastest way to get a random sample for large entries
        sample = pd.read_sql_query('select * from '+str(month)+' order by random() limit '+str(number),conn)
    
    #Turns the dropoff_datetime column into an array of time values
    tmpdate = sample['dropoff_datetime'].get_values()
    
    #Initiliazing three arrays that will be our new columns in our sample
    dropoff_hour = [0]*(number)
    dropoff_day = [0]*(number)
    day_of_week = [0]*(number)
    
    #Loop that will strip the time from every entry in tmpdate and assign it to the right column
    for i, date in enumerate(tmpdate):
        tmp = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        dropoff_hour[i] = tmp.hour
        dropoff_day[i] = tmp.day
        day_of_week[i] = tmp.weekday()
    
    #Creating the new columns for our sample
    sample['dropoff_hour'] = dropoff_hour
    sample['dropoff_day'] = dropoff_day
    sample['weekday'] = day_of_week
    
    #Returning the sample
    return sample

In [13]:
#Example of get_sample
x = get_sample(2013,"december",5)

#This will print out the sample to a csv file so it can be loaded into JMP or other software
x.to_csv('test_sample3.csv')


  vendor_id      pickup_datetime     dropoff_datetime  passenger_count  \
0       VTS  2013-11-04 12:22:00  2013-11-04 12:57:00                5   
0       VTS  2013-11-06 11:51:00  2013-11-06 12:00:00                3   
0       CMT  2013-11-04 12:23:57  2013-11-04 12:36:11                1   
0       VTS  2013-11-21 13:26:00  2013-11-21 13:37:00                1   
0       VTS  2013-11-10 13:54:00  2013-11-10 14:29:00                1   

   trip_distance  pickup_longitude  pickup_latitude  rate_code  \
0          18.53        -73.961470        40.796157          2   
0           1.70        -73.958840        40.799562          1   
0           5.00        -74.004530        40.707150          1   
0           1.38        -73.990215        40.752067          1   
0          21.21        -73.971610        40.795270          3   

  store_and_fwd_flag  dropoff_longitude   ...     payment_type fare_amount  \
0                            -73.789962   ...              CSH        52.0   
0 